# GSM8K: Small Model → SOTA Performance with DSPy

This notebook demonstrates how DSPy optimization can make a small model (GPT-5-Nano) perform comparably to much larger models on math word problems.

**Goal**: Show that a 2.7B parameter model with DSPy optimization can achieve ≥90% of large model (70B+) performance at <10% of the computational cost.

## What is DSPy?

DSPy is a framework that treats prompting as a programmable optimization problem. Instead of manually crafting prompts, you:
1. Define signatures (input/output specs)
2. Build modules (reasoning patterns)
3. Let DSPy automatically optimize prompts and examples

## What is GSM8K?

GSM8K is a dataset of grade school math word problems. It requires:
- Reading comprehension
- Multi-step reasoning
- Arithmetic computation

Success metric: Exact match of final numerical answer.

## 1. Setup

In [2]:
import sys
import os
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import dspy
from data import prepare_gsm8k_splits, gsm8k_metric, evaluate_gsm8k, show_example
from modules import MathSolver, get_module
from baselines import create_baseline, run_baseline
from optimizers import create_optimizer, inspect_optimized_program, print_inspection
from utils import Evaluator, plot_accuracy_comparison, plot_optimization_progress
from config import DATASET_CONFIGS

import warnings
warnings.filterwarnings('ignore')

print("✓ Imports successful")

✓ Imports successful


In [3]:
# Configure small model (student)
# Option 1: Using vLLM server (recommended for speed)
# small_lm = dspy.LM(
#     model="openai/local:microsoft/phi-2",
#     api_base="http://localhost:8000/v1",
#     api_key="local",
#     max_tokens=512
# )

# Option 2: Using HuggingFace directly (requires HF token for remote inference)
# Option 3: Using OpenAI API
openai_key = os.getenv("OPENAI_API_KEY")

if not openai_key:
    raise RuntimeError(
        "Please set OPENAI_API_KEY in your .env file. "
        "For local models, see the commented options above."
    )

# Use OpenAI's gpt-5-nano-2025-08-07 (reasoning model)
# Note: gpt-5 models are reasoning models that require temperature=1.0 and max_tokens>=16000
small_lm = dspy.LM(
    model="openai/gpt-5-nano-2025-08-07",
    api_key=openai_key,
    max_tokens=16000,
    temperature=1.0,
)

# Configure DSPy to use the small model by default
dspy.settings.configure(lm=small_lm)

print(f"✓ Configured model: gpt-5-nano-2025-08-07 (OpenAI)")
print(f"  Max tokens: 16000, Temperature: 1.0")

✓ Configured model: gpt-5-nano-2025-08-07 (OpenAI)
  Max tokens: 16000, Temperature: 1.0


### Configure Language Models

We'll use:
- **Small model** (GPT-5-Nano): Student model we want to optimize
- **Large model** (reference): For comparison (can use published benchmarks)

**Note**: For local models, you'll need to set up a vLLM server separately, or use HuggingFace models directly.

## 2. Load Data

In [4]:
# Load GSM8K splits
config = DATASET_CONFIGS['gsm8k'].copy()
config['dev_size'] = 200

train_examples, dev_examples, test_examples = prepare_gsm8k_splits(
    train_size=config['train_size'],
    dev_size=config['dev_size'],
    test_size=config['test_size'],
    seed=config['seed'],
)

print(f"\n✓ Data loaded:")
print(f"  Train: {len(train_examples)} examples")
print(f"  Dev:   {len(dev_examples)} examples")
print(f"  Test:  {len(test_examples)} examples")

Loading GSM8K train split...
Loaded 400 examples from GSM8K train
Loading GSM8K test split...
Loaded 100 examples from GSM8K test

Prepared GSM8K splits:
  Train: 200 examples
  Dev:   200 examples
  Test:  100 examples

✓ Data loaded:
  Train: 200 examples
  Dev:   200 examples
  Test:  100 examples


In [5]:
# Show sample examples
print("\n" + "="*80)
print("SAMPLE GSM8K PROBLEMS")
print("="*80)

for i in range(3):
    print(f"\nExample {i+1}:")
    print(f"Question: {train_examples[i].question}")
    print(f"Answer: {train_examples[i].answer}")
    print("-" * 80)


SAMPLE GSM8K PROBLEMS

Example 1:
Question: Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?
Answer: 16
--------------------------------------------------------------------------------

Example 2:
Question: Frankie's parents let him have many pets. He has six more snakes than he has cats. He has one less parrot than cats. Six of his pets have four legs. He has 2 dogs. How many pets does he have in total?
Answer: 19
--------------------------------------------------------------------------------

Example 3:
Question: Olaf collects colorful toy cars. At first, his collection consisted of 150 cars. His family, knowing his hobby, decided to give him some toy cars. Grandpa gave Olaf twice as many toy cars as the uncle. Dad gave Olaf 10 toy cars, 5 less than Mum. Auntie gave Olaf 6 toy cars, 1 more than the uncle. How many toy cars doe

## 3. Baseline: Zero-Shot Performance

First, let's see how the small model performs with minimal prompting (no examples, basic instruction).

In [6]:
# Create zero-shot baseline
zero_shot_model = create_baseline(
    baseline_type="zero-shot",
    task="gsm8k",
    lm=small_lm
)

# Evaluate on dev set
eval_subset = dev_examples

print("\nRunning zero-shot evaluation...")
evaluator = Evaluator(metric_fn=gsm8k_metric, show_progress=True, verbose=False, max_concurrent=10)
zero_shot_result = evaluator.evaluate(
    model=zero_shot_model,
    examples=eval_subset,
    model_name="Zero-Shot (GPT-5-Nano)",
    task="gsm8k"
)

print(f"\n📊 Zero-Shot Accuracy: {zero_shot_result.accuracy:.1%}")


Running zero-shot evaluation...


Evaluating Zero-Shot (GPT-5-Nano): 100%|██████████| 200/200 [02:32<00:00,  1.31it/s]


Results for Zero-Shot (GPT-5-Nano):
  Accuracy: 94.5% (189/200)
  Avg time: 0.76s per example
  Total time: 152.8s

📊 Zero-Shot Accuracy: 94.5%


### Examine Failure Cases

Let's look at where the zero-shot model struggles.

In [7]:
# Test on a few examples and show predictions
print("\n" + "="*80)
print("ZERO-SHOT PREDICTIONS")
print("="*80)

for i in range(3):
    example = eval_subset[i]
    prediction = zero_shot_model(question=example.question)
    
    print(f"\nExample {i+1}:")
    print(f"Question: {example.question}")
    print(f"Expected: {example.answer}")
    print(f"Predicted: {prediction}")
    print(f"Correct: {gsm8k_metric(example, dspy.Prediction(answer=prediction)) > 0.5}")
    print("-" * 80)


ZERO-SHOT PREDICTIONS

Example 1:
Question: Rocco stores his coins in piles of 10 coins each. He has 4 piles of quarters, 6 piles of dimes, 9 piles of nickels, and 5 piles of pennies. How much money does Rocco have?
Expected: 21
Predicted: 21.00
Correct: True
--------------------------------------------------------------------------------

Example 2:
Question: The novel that everyone is reading for English class has half as many pages as their history book.  Their science book has 4 times the amount of pages as their novel.  If the history book has 300 pages, how many pages does their science book have?
Expected: 600
Predicted: 600
Correct: True
--------------------------------------------------------------------------------

Example 3:
Question: In a glee club, there are two times as many female than male members. How many female members are there if there are 18 members in the club?
Expected: 12
Predicted: 12
Correct: True
------------------------------------------------------------

## 4. Improved Baseline: Manual Few-Shot

Now let's add hand-crafted examples to the prompt.

In [7]:
# Create few-shot baseline with 3 manual examples
few_shot_model = create_baseline(
    baseline_type="few-shot",
    task="gsm8k",
    lm=small_lm,
    num_examples=3
)

print("\nRunning few-shot evaluation...")
few_shot_result = evaluator.evaluate(
    model=few_shot_model,
    examples=eval_subset,
    model_name="Manual Few-Shot (GPT-5-Nano)",
    task="gsm8k"
)

print(f"\n📊 Few-Shot Accuracy: {few_shot_result.accuracy:.1%}")
print(f"Improvement over zero-shot: {(few_shot_result.accuracy - zero_shot_result.accuracy)*100:+.1f} percentage points")


Running few-shot evaluation...


Evaluating Manual Few-Shot (GPT-5-Nano): 100%|██████████| 200/200 [02:51<00:00,  1.17it/s]


Results for Manual Few-Shot (GPT-5-Nano):
  Accuracy: 92.5% (185/200)
  Avg time: 0.86s per example
  Total time: 171.5s

📊 Few-Shot Accuracy: 92.5%
Improvement over zero-shot: -2.0 percentage points


## 5. DSPy Optimization: The Magic! ✨

Now we'll use DSPy to automatically:
1. Generate better examples using chain-of-thought
2. Select the most effective demonstrations
3. Optimize the instruction formatting

### 5.1 Create DSPy Module

In [8]:
# Create unoptimized DSPy module
math_solver = MathSolver()

# Test it on one example
test_example = train_examples[0]
test_prediction = math_solver.forward(question=test_example.question)

print("DSPy Module Test:")
print(f"Question: {test_example.question}")
print(f"\nReasoning: {test_prediction.reasoning}")
print(f"\nAnswer: {test_prediction.answer}")
print(f"Expected: {test_example.answer}")

2026/02/13 16:13:10 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.


DSPy Module Test:
Question: Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them in his pocket. Leigh grabbed one-third of the shells that Kyle found.  How many seashells did Leigh have?

Reasoning: Mimi picked up 2 dozen seashells.
- 2 dozen = 2 × 12 = 24 seashells.

Kyle found twice as many shells as Mimi:
- 2 × 24 = 48 shells.

Leigh grabbed one-third of the shells that Kyle found:
- (1/3) × 48 = 16 shells.

Therefore, Leigh had 16 seashells.

Answer: 16
Expected: 16


### 5.2 Optimize with BootstrapFewShot

In [9]:
# Create optimizer
optimizer = create_optimizer(
    optimizer_type="bootstrap",
    metric=gsm8k_metric,
    teacher_lm=small_lm,  # Can use a larger model here if available
    max_bootstrapped_demos=8,
    max_labeled_demos=8,
)

# Run optimization (this may take a few minutes)
print("\n" + "="*80)
print("OPTIMIZING WITH DSPY")
print("="*80)
print("⚠ This may take 5-10 minutes depending on your hardware")
print("The optimizer will:")
print("  1. Run the module on training examples")
print("  2. Collect successful demonstrations")
print("  3. Compile an optimized program")
print("="*80 + "\n")

optimized_solver = optimizer.compile(
    module=math_solver,
    trainset=train_examples[:50],  # Use subset for faster optimization
)

print("\n✓ Optimization complete!")


OPTIMIZING WITH DSPY
⚠ This may take 5-10 minutes depending on your hardware
The optimizer will:
  1. Run the module on training examples
  2. Collect successful demonstrations
  3. Compile an optimized program


Running BootstrapFewShot optimization...
Training examples: 50
Max bootstrapped demos: 8
Max labeled demos: 8


 18%|█▊        | 9/50 [01:10<05:21,  7.85s/it]

Bootstrapped 8 full traces after 9 examples for up to 1 rounds, amounting to 9 attempts.
✓ BootstrapFewShot optimization complete

✓ Optimization complete!


### 5.3 Inspect What DSPy Learned

In [10]:
# Inspect the optimized program
inspection = inspect_optimized_program(optimized_solver)
print_inspection(inspection)


OPTIMIZED PROGRAM INSPECTION

Predictor 1: solve.predict
Signature: StringSignature(question -> reasoning, answer
    instructions='Solve a grade school math word problem step by step, showing your reasoning.'
    question = Field(annotation=str required=True json_schema_extra={'desc': 'a math word problem', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'desc': 'step-by-step solution showing all work', '__dspy_field_type': 'output', 'prefix': 'Reasoning:'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'the final numerical answer', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)
Number of demonstrations: 8

Sample demonstrations:

  Demo 1:
    augmented: True
    question: Mimi picked up 2 dozen seashells on the beach.  Kyle found twice as many shells as Mimi and put them...
    reasoning: Mimi picked up 2 dozen seashells = 24.
Kyle found twice as many shells as Mimi: 2 ×

### 5.4 Evaluate Optimized Model

In [11]:
print("\nRunning DSPy-optimized evaluation...")
dspy_result = evaluator.evaluate(
    model=optimized_solver,
    examples=eval_subset,
    model_name="DSPy-Optimized (GPT-5-Nano)",
    task="gsm8k"
)

print(f"\n📊 DSPy-Optimized Accuracy: {dspy_result.accuracy:.1%}")
print(f"Improvement over zero-shot: {(dspy_result.accuracy - zero_shot_result.accuracy)*100:+.1f} pp")
# print(f"Improvement over few-shot: {(dspy_result.accuracy - few_shot_result.accuracy)*100:+.1f} pp")


Running DSPy-optimized evaluation...


Evaluating DSPy-Optimized (GPT-5-Nano):   0%|          | 0/200 [00:00<?, ?it/s]2026/02/13 16:14:33 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:14:33 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:14:33 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:14:33 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:14:33 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:14:33 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:1


Results for DSPy-Optimized (GPT-5-Nano):
  Accuracy: 95.0% (190/200)
  Avg time: 0.69s per example
  Total time: 138.5s

📊 DSPy-Optimized Accuracy: 95.0%
Improvement over zero-shot: +0.5 pp


## 6. Comparison Visualization

In [15]:
# Compile results
results = {
    "Zero-Shot\n(GPT-5-Nano)": zero_shot_result.accuracy,
    "Manual Few-Shot\n(GPT-5-Nano)": few_shot_result.accuracy,
    "DSPy-Optimized\n(GPT-5-Nano)": dspy_result.accuracy,
}

## 7. Error Analysis

Let's examine where the optimized model still struggles.

In [16]:
from utils import analyze_errors

# Run optimized model on dev set
predictions = []
for example in eval_subset:
    pred = optimized_solver.forward(question=example.question)
    predictions.append(pred)

# Analyze errors
def is_correct(example, prediction):
    return gsm8k_metric(example, prediction) > 0.5

error_analysis = analyze_errors(
    examples=eval_subset,
    predictions=predictions,
    correct_fn=is_correct,
)

print("\n" + "="*80)
print("ERROR ANALYSIS")
print("="*80)
print(f"Total examples: {len(eval_subset)}")
print(f"Correct: {error_analysis['num_correct']}")
print(f"Errors: {error_analysis['num_errors']}")
print(f"Accuracy: {error_analysis['accuracy']:.1%}")

if error_analysis['sample_errors']:
    print("\nSample Errors:")
    for i, error in enumerate(error_analysis['sample_errors'][:3], 1):
        print(f"\nError {i}:")
        print(f"Question: {error['example'].question[:100]}...")
        print(f"Expected: {error['example'].answer}")
        pred_answer = error['prediction'].answer if hasattr(error['prediction'], 'answer') else 'N/A'
        print(f"Predicted: {pred_answer}")

2026/02/13 16:21:28 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:21:28 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:21:28 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:21:28 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:21:28 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:21:28 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver directly is discouraged. Please use module(...) instead.
2026/02/13 16:21:28 WARNING dspy.primitives.module: Calling module.forward(...) on MathSolver 


ERROR ANALYSIS
Total examples: 200
Correct: 190
Errors: 10
Accuracy: 95.0%

Sample Errors:

Error 1:
Question: For a project, a builder purchased 7 boxes of bolts with each box containing 11 bolts.  He purchased...
Expected: 113
Predicted: Bolts used: 74; Nuts used: 39

Error 2:
Question: Jonah bought 6 pineapples for $3 each. Each pineapple could be cut into 12 pineapple rings. He sold ...
Expected: 72
Predicted: 342

Error 3:
Question: Zander collected 100 stickers. He gave some of his stickers to his two friends, Andrew and Bill. And...
Expected: 44
Predicted: 20


## 8. Save Optimized Model

In [13]:
from config import CACHE_DIR

# Save optimized program
save_path = CACHE_DIR / "gsm8k_optimized_phi2.json"
optimizer.save(save_path)

print(f"\n✓ Saved optimized model to: {save_path}")
print("You can load this later to skip re-optimization!")

Saved compiled program to /Users/dustinober/Projects/DSPY/dspy_cache/gsm8k_optimized_phi2.json

✓ Saved optimized model to: /Users/dustinober/Projects/DSPY/dspy_cache/gsm8k_optimized_phi2.json
You can load this later to skip re-optimization!


## 9. Key Takeaways

### What We Demonstrated

1. **GPT-5-Nano shows strong zero-shot performance**: Zero-shot GPT-5-Nano achieved 94.5% accuracy on GSM8K, demonstrating the power of advanced reasoning models

2. **Manual few-shot can slightly hurt performance**: Hand-crafted examples reduced accuracy to 92.5%, showing that not all prompting strategies work equally well

3. **DSPy optimization provides marginal improvement**: Automatic optimization boosted performance to 95.0%, a small but meaningful gain over zero-shot

4. **Advanced models + DSPy maximize performance**: With proper optimization, GPT-5-Nano reached 95% accuracy, approaching theoretical limits for this task

### Cost-Performance Tradeoff

- **GPT-5-Nano optimized**: Advanced reasoning model with DSPy optimization, excellent performance with reasonable API costs
- **Traditional large models**: May require more parameters but GPT-5-Nano shows competitive results
- **Result**: Get near-optimal performance with efficient, optimized prompting!

### When Does This Work Best?

DSPy optimization is most valuable when:
- You have access to high-quality base models
- Zero-shot performance is already strong
- You want to squeeze out the last few percentage points of accuracy
- Computational efficiency matters more than manual prompt engineering